In [1]:
# restarts the service
#!sudo supervisorctl stop core
#!sudo supervisorctl start core

In [2]:
#!tail -n 20 /home/ebrahim/core.out

In [3]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow" # Xception needs TF

import requests
from PIL import Image
import io
import re
import urllib.parse

import numpy as np

from keras.applications import ResNet50
from keras.applications import InceptionV3
from keras.applications import Xception # TensorFlow ONLY
from keras.applications import VGG16
from keras.applications import VGG19
from keras.applications import MobileNet
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
import keras.applications.inception_v3

Using TensorFlow backend.


In [4]:
MODELS = {
    "mobilenet": MobileNet,
    "vgg16": VGG16,
    "vgg19": VGG19,
    "inception": InceptionV3,
    "xception": Xception, # TensorFlow ONLY
    "resnet": ResNet50
}

AVALABLE_MODELS = {}
#available_models = {model_name: model(weights="imagenet") for model_name, model in MODELS.items()}

def img_classify(img, model_name="vgg19"):
    inputShape = (224, 224)
    preprocess = imagenet_utils.preprocess_input

    if model_name in ("inception", "xception"):
        inputShape = (299, 299)
        preprocess = keras.applications.inception_v3.preprocess_input

    if model_name not in AVALABLE_MODELS:
        AVALABLE_MODELS[model_name] = MODELS[model_name](weights="imagenet")
    model = AVALABLE_MODELS[model_name]

    img = img_to_array(img.convert('RGB').resize(inputShape))
    # prepend a numpy array dimension, (x, y, 3) -> (1, x, y, 3)
    img = preprocess(np.expand_dims(img, axis=0))

    preds = model.predict(img)
    P = imagenet_utils.decode_predictions(preds)

    return [[label, float(prob), imagenetID] for (imagenetID, label, prob) in P[0]]

In [12]:
def load_pages(pages):
    infos = requests.get('https://commons.wikimedia.org/w/api.php',
                 {'action': 'query', 'prop': 'revisions', 'rvprop': 'content', 'format': 'json',
                  'titles': '|'.join(pages)}).json()

    return {x['title']: x['revisions'][0]['*'] for x in infos['query']['pages'].values()}


def filter_categories(cats):
    return {title for title, content in load_pages(cats).items()
            if 'catredir' not in content and
               'Category redirect' not in content}


def search_for_category(term):
    # https://commons.wikimedia.org/w/api.php?action=help&modules=query%2Bsearch
    info = requests.get('https://commons.wikimedia.org/w/api.php',
                        {'action': 'query', 'list': 'search', 'srsearch': term,
                         'srprop': '', 'srnamespace': 14, 'format': 'json'},
                        headers={'User-Agent': 'Deep Learning Services'}).json()
    cats = [x['title'] for x in info['query']['search']]
    return list(filter_categories(cats)) if cats else []


def image_category(image, model):
    #if model not in MODELS:
    #    raise Exception('Requested model not available')
    infos = requests.get('https://commons.wikimedia.org/w/api.php',
                         {'action': 'query', 'prop': 'imageinfo', 'format': 'json',
                          'iiprop': 'url', 'iiurlwidth': 300,
                          'titles': 'File:' + image},
                         headers={'User-Agent': 'Deep Learning Services'}).json()['query']['pages']
    url = ((list(infos.values()) or [{}])[0].get('imageinfo') or [{}])[0].get('thumburl')
    
    if not url:
        return []
    
    tags = img_classify(Image.open(io.BytesIO(requests.get(url).content)))
    
    categories_from_tagger = search_for_category(tags[0][0].replace('_', ' '))
    categories_from_name = search_for_category(image.replace('_', ' ').replace('.', ' ').split(' ')[0])
    
    result = [x for chain in zip(categories_from_tagger) for x in chain] + categories_from_tagger + categories_from_name
    
    return {
        'name': image,
        'tags': tags,
        'categories': list(set(list(result)[:5]))
    }


image_category('Yipu 4.jpg', 'xception')

{'categories': ['Category:Patio de los Naranjos',
  'Category:Patio del Mexuar',
  'Category:Patio de los Naranjos, Mosque-Cathedral of Córdoba',
  'Category:Patios in the Philippines',
  'Category:Patio de los Arrayanes'],
 'name': 'Yipu 4.jpg',
 'tags': [['patio', 0.3328576982021332, 'n03899768'],
  ['lakeside', 0.19316774606704712, 'n09332890'],
  ['boathouse', 0.15833984315395355, 'n02859443'],
  ['valley', 0.07808777689933777, 'n09468604'],
  ['greenhouse', 0.03299791365861893, 'n03457902']]}